In [25]:
import os
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--disable-gpu') 

driver = webdriver.Chrome(executable_path='/Users/alex/chromedriver', options=chrome_options)

In [28]:
## make chromedriver run on GCE
os.environ['webdriver.chrome.driver'] = "/usr/bin/chromedriver"
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument('--disable-gpu') 
chrome_options.add_argument("start-maximized")
chrome_options.add_argument("disable-infobars") 
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox") 
driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', options=chrome_options)

In [2]:
def get_html(path, use_selenium):
    if use_selenium:
        driver.get(path)
        return driver.page_source
    else:
        page = requests.get(path)
        return page.text

def parse_table(soup):
    table_output = []
    for row in soup.find_all('tr'):
        row_ouput = []
        for cell in row.find_all('td'):
            if cell.find('a'):
                cell_link = cell.a['href']
                row_ouput.append(cell_link)
            cell_value = ''.join(cell.stripped_strings)
            row_ouput.append(cell_value)
        table_output.append(row_ouput)

    table_df = (pd.DataFrame(
        list(filter(lambda x: len(x) != 0, table_output)), 
        columns=['newssniff_link', 'headline', 'version', 'outlet', 'last_updated'])
    )
    return table_df

def get_search_result(page, use_selenium=True):
    url = 'https://www.newssniffer.co.uk/versions?page=%s' % page
    search_page = get_html(url, use_selenium=True)
    soup = BeautifulSoup(search_page.text)
    page_df = parse_table(soup)
    page_df['search_page'] = page
    return page, page_df


def parse_article(html):
    """Given the HTML of an article return a table with a row for each version."""

    ## get url
    soup = BeautifulSoup(html)
    source, url = soup.find_all('cite')[0].text, soup.find_all('cite')[1].text

    ## get the rest of the article
    version_tables = pd.read_html(html)[0]
    version_table_flat = (version_tables
     .apply(lambda s: ('</p><p>').join(s.dropna().tolist()))
     .reset_index()
     .rename(columns={'level_0': 'version', 'level_1':'title', 'level_2': 'time', 0:'text'})
    )

    ## merge
    version_table_flat['url'] = url
    version_table_flat['source'] = source

    return version_table_flat

In [ ]:
last_page = 239179
start_page = 50503
all_search_output = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as pool:        
    result_futures = list(map(lambda x: pool.submit(get_search_result, x), range(start_page, last_page)))
    for future in tqdm(concurrent.futures.as_completed(result_futures), total=last_page - start_page):
        try:
            page_idx, page_df = future.result()
            all_search_output.append(page_df)
            if len(all_search_output) > 1000:
                all_search_output_df = pd.concat(all_search_output)
                all_search_output_df.to_csv('cache/newssniffer-index-%s.csv' % page_idx)
                all_search_output = []
        except Exception as e:
            print('e is', e, type(e))

In [7]:
pd.concat(all_search_output).to_csv('cache/newssniffer-index-%s.csv' % page_idx)

In [53]:
search_pages = list(map(lambda x: pd.read_csv(x, usecols=['search_page'], squeeze=True), search_files))
searched_df = pd.concat(search_pages) 

In [65]:
start_page = 200
last_page = 239295
all_search_pages = pd.Series(list(range(start_page, last_page)))

In [78]:
ls ../scraping/scrapy-cloud/newssniffer_scrape/output_dir/

file-ids-scraped.csv      search-pages-scraped.csv


In [77]:
searched_df.drop_duplicates().to_csv('../scraping/scrapy-cloud/newssniffer_scrape/output_dir/search-pages-scraped.csv')

In [90]:
search_files = glob.glob('../scraping/scrapy-cloud/output_dir/newssniffer*')
search_pages = list(map(lambda x: pd.read_csv(x, index_col=0), search_files))

In [91]:
pd.concat(search_pages)

,newssniff_link,headline,version,outlet,last_updated,search_page
0,https://www.newssniffer.co.uk/articles/1709190...,Fourth Soldier Dies After Coordinated Bomb Bla...,1,nytimes,"04 Dec 2018, 05:50",52505
1,https://www.newssniffer.co.uk/articles/1709318...,Arrests over India policeman killed by 'cow sl...,1,bbc,"04 Dec 2018, 05:50",52505
2,https://www.newssniffer.co.uk/articles/1695611...,Left behind: families face uncertain future wh...,2,guardian,"04 Dec 2018, 05:47",52505
3,https://www.newssniffer.co.uk/articles/1709331...,Country diary: thatching is a job to take your...,0,guardian,"04 Dec 2018, 05:45",52505
4,https://www.newssniffer.co.uk/articles/1689299...,Angela Merkel’s successor could be bad news fo...,1,guardian,"04 Dec 2018, 05:45",52505
...,...,...,...,...,...,...
11,https://www.newssniffer.co.uk/articles/1680196...,Elizabeth Warren Releases DNA Results on Nativ...,0,nytimes,"15 Oct 2018, 13:55",56508
12,https://www.newssniffer.co.uk/articles/1680150...,Fracking starts at landmark Lancashire site,2,bbc,"15 Oct 2018, 13:55",56508
13,https://www.newssniffer.co.uk/articles/1680175...,"Cologne hostage taker ‘under control,’ one wom...",1,rtcom,"15 Oct 2018, 13:55",56508
14,https://www.newssniffer.co.uk/articles/1679806...,Among the Ruins of Mexico Beach Stands One Hou...,3,nytimes,"15 Oct 2018, 13:55",56508


# See output so far

In [6]:
import glob
files = list(map(
    lambda x: pd.read_csv(x, index_col=0),
    glob.glob('cache/newssniffer-index-*')
))

files_df = (pd.concat(files)
            .reset_index(drop=True)
            .assign(file_id=lambda df: df['newssniff_link'].str.split('/').str.get(4))
           )

max_diff_per_file_id = (
    files_df[['version', 'file_id']]
    .assign(version=lambda df: df['version'].astype(int))
    .groupby('file_id')
    ['version']    
    .max()
)

files_df_versions = files_df.merge(
    max_diff_per_file_id.astype(int).to_frame('num_diffs_per_file'),
    left_on='file_id',
    right_index=True
)

In [ ]:
files_df['file_id'].value_counts().head()
files_df.drop_duplicates('file_id')['outlet'].value_counts()
files_df['last_updated_time'] = files_df['last_updated'].pipe(lambda s: pd.to_datetime(s, errors='coerce'))
files_df['last_updated_time'].dropna().sort_values().min()
files_df_versions.groupby('outlet')['num_diffs_per_file'].median()
files_df_versions.drop_duplicates('file_id')['outlet'].value_counts()

# Test Running the File fetcher

In [8]:
min_cut, max_cut = (
    files_df_versions
        .drop_duplicates('file_id')['num_diffs_per_file']
        .loc[lambda s: s != 0].quantile([.01, .99])
)

to_scrape = (files_df_versions
 .loc[lambda df: df['num_diffs_per_file'].pipe(lambda s: (s >= min_cut) & (s<=max_cut))] ## 1, 30
 [['file_id', 'num_diffs_per_file']]
 .drop_duplicates()
)

In [24]:
import time

In [ ]:
article_dfs = []
num_tries = 5
start_at = 11194
for loop_idx, (file_id, num_diffs) in tqdm(
    enumerate(to_scrape.iloc[start_at:].itertuples(index=False)), 
    total=len(to_scrape.iloc[start_at:])
):
    if (loop_idx % 500 == 0) and len(article_dfs) > 0:
        to_disk = pd.concat(article_dfs)
        to_disk.to_csv('cache/article-versions-%s.csv' % (loop_idx + start_at))
        article_dfs = []
    
    v = list(range(num_diffs+1))
    for s, e in list(zip(v[:-1], v[1:])):
        for i in range(num_tries):
            try:
                url = 'https://www.newssniffer.co.uk/articles/%s/diff/%s/%s' % (file_id, s, e)
                html = get_html(url, use_selenium=True)
                article_df = parse_article(html)
                article_df['file_id'] = file_id
                article_dfs.append(article_df)
                break
            except ConnectionError:
                print('driver failed... re-initializing...')
                driver.quit()
                driver = webdriver.Chrome(executable_path='/Users/alex/chromedriver', options=chrome_options)
            except:
                print('failed, sleeping...')
                time.sleep(5)

# Article-Level Parsing

In [ ]:
import pandas as pd 
html = pd.read_html('https://www.newssniffer.co.uk/articles/2057481/diff/10/11')
html[0]

In [73]:
import glob
import re
import os
import concurrent.futures

In [ ]:
import concurrent.futures
iterable = [1,2,3,4,6,7,8,9,10]

def f(x):
    if x == 2:
        raise Exception('x')
    return x

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:


In [72]:
import time
e = ThreadPoolExecutor(4)
s = range(10)
for i in e.map(time.sleep, s):
    print(i)

None
None
None
None
None
None
None
None
None
None


# Docker

In [ ]:
import scrapy
from selenium import webdriver

class DemoSpider(scrapy.Spider):
    name = 'demo'
    start_urls = ['http://quotes.toscrape.com/js']

    def __init__(self, *args, **kwargs):
        super(DemoSpider, self).__init__(*args, **kwargs)

        options = webdriver.ChromeOptions()
        options.add_argument("--disable-extensions")
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        self.driver = webdriver.Chrome(chrome_options=options, executable_path='/usr/bin/chromedriver')
    
    def parse(self, response):
        self.driver.get(response.url)
        for quote in self.driver.find_elements_by_css_selector('div.quote'):
            yield {
                'quote': quote.find_element_by_css_selector('span').text,
                'author': quote.find_element_by_css_selector('small').text,
            }
        next_page_url = response.css('nav li.next a ::attr(href)').extract_first()
        if next_page_url:
            yield scrapy.Request(response.urljoin(next_page_url))

In [94]:
t = """<html><head>\n  \t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t  <title>'The ordinary Africans hitting the headlines in 2016' article at News Sniffer - News Sniffer</title>\n\t  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">\n    <link rel="stylesheet" media="all" href="/assets/application-c552a79478ab8665fb5265320f163a84fc6eb1ad82acdd97cbf264e4ac904b2c.css">\n        <link rel="alternate" type="application/rss+xml" title="RSS feed of versions of this article" href="https://www.newssniffer.co.uk/articles/1297019.rss">\n    <link href="http://www.bbc.co.uk/news/world-africa-38473883" rel="external alternate author" title="Original article">\n\t<meta name="generator" content="BbcNewsPageParserV5">\n\n    <script src="//code.jquery.com/jquery-3.1.0.slim.min.js" integrity="sha256-cRpWjoSOw5KcyIOaZNo4i6fZ9tKPhYYb6i5T9RSVJG8=" crossorigin="anonymous"></script>\n\t  <script src="/assets/application-1f49f9044073b7b6a5ec036504202e9f7f93f432c76ecccf386831ccfc644a8c.js"></script>\n</head>\n<body>\n\t<header>\n\t\t<div id="header" class="header">\n\t\t\t<div class="logo-head">\n\t\t\t\t<a href="/"><img src="/images/newsniffer-text.png" alt="News Sniffer" class="logo" id="logo"></a>\n\t\t\t</div>\n\t\t\t<div class="toggle-nav">\n\t            <span class="bar"></span>\n\t            <span class="bar"></span>\n\t            <span class="bar"></span>\n             </div>\n\t\t\t<nav class="navigation">\n\t\t\t\t<ul id="nav" class="menu">\n\t\t\t\t\t<li><a title="News article Revisions" class="" href="https://www.newssniffer.co.uk/versions">News Article Revisions</a></li>\n\t\t\t\t\t<li><a href="/blog/">Blog</a></li>\n\t\t\t\t\t<li><a href="/blog/about">About</a></li>\n\t\t\t\t</ul>\n\t\t\t</nav>\n\t\t</div>\n\t</header>\n\t\n\t<div id="content" class="content">\n\n\t<h1>The ordinary Africans hitting the headlines in 2016</h1>\n\n<p>This article is from the source '<cite>bbc</cite>' and was first published or seen\non <time datetime="2016-12-30T22:54:05Z">December 30, 2016 22:54 (UTC)</time>.\n\nIt last changed over 40 days ago and won't be checked again for changes.\n\n</p><p>You can find the current article at its original source at <cite class="article_url"><a rel="alternate external author" href="http://www.bbc.co.uk/news/world-africa-38473883">http://www.bbc.co.uk/news/world-africa-38473883</a></cite></p>\n\n<p>The article has changed <var>0</var> times. There is an <a class="feed rss" rel="alternate" href="https://www.newssniffer.co.uk/articles/1297019.rss">RSS feed of changes</a> available.</p>\n\n<table>\n<tbody><tr>\n  <th class="version">Version</th>\n  <th class="created_at">Created at</th>\n\t<th class="period">Period</th>\n  <th class="title">Title</th>\n</tr>\n</tbody></table>\n\n\t</div>\n\n\t<footer>\n\t\t<div id="footer">\n\t\t\t\t<p>Currently monitoring 1,792,836 news articles \n\t\t\t\twith 3,829,486 different versions.</p>\n\t\t</div>\n\t</footer>\n\n\n</body></html>"""

In [95]:
from IPython.display import display, HTML

In [97]:
display(HTML(t))

Version,Created at,Period,Title


In [98]:
t = """<html><head>\n  \t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t  <title>'Costa Concordia salvage – interactive guide' article at News Sniffer - News Sniffer</title>\n\t  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">\n    <link rel="stylesheet" media="all" href="/assets/application-c552a79478ab8665fb5265320f163a84fc6eb1ad82acdd97cbf264e4ac904b2c.css">\n        <link rel="alternate" type="application/rss+xml" title="RSS feed of versions of this article" href="https://www.newssniffer.co.uk/articles/518330.rss">\n    <link href="http://www.guardian.co.uk/world/interactive/2012/may/18/costa-concordia-salvage-interactive" rel="external alternate author" title="Original article">\n\t<meta name="generator" content="GuardianPageParserV1">\n\n    <script src="//code.jquery.com/jquery-3.1.0.slim.min.js" integrity="sha256-cRpWjoSOw5KcyIOaZNo4i6fZ9tKPhYYb6i5T9RSVJG8=" crossorigin="anonymous"></script>\n\t  <script src="/assets/application-1f49f9044073b7b6a5ec036504202e9f7f93f432c76ecccf386831ccfc644a8c.js"></script>\n</head>\n<body>\n\t<header>\n\t\t<div id="header" class="header">\n\t\t\t<div class="logo-head">\n\t\t\t\t<a href="/"><img src="/images/newsniffer-text.png" alt="News Sniffer" class="logo" id="logo"></a>\n\t\t\t</div>\n\t\t\t<div class="toggle-nav">\n\t            <span class="bar"></span>\n\t            <span class="bar"></span>\n\t            <span class="bar"></span>\n             </div>\n\t\t\t<nav class="navigation">\n\t\t\t\t<ul id="nav" class="menu">\n\t\t\t\t\t<li><a title="News article Revisions" class="" href="https://www.newssniffer.co.uk/versions">News Article Revisions</a></li>\n\t\t\t\t\t<li><a href="/blog/">Blog</a></li>\n\t\t\t\t\t<li><a href="/blog/about">About</a></li>\n\t\t\t\t</ul>\n\t\t\t</nav>\n\t\t</div>\n\t</header>\n\t\n\t<div id="content" class="content">\n\n\t<h1>Costa Concordia salvage – interactive guide</h1>\n\n<p>This article is from the source '<cite>guardian</cite>' and was first published or seen\non <time datetime="2012-05-18T18:40:05Z">May 18, 2012 18:40 (UTC)</time>.\n\nIt last changed over 40 days ago and won't be checked again for changes.\n\n</p><p>You can find the current article at its original source at <cite class="article_url"><a rel="alternate external author" href="http://www.guardian.co.uk/world/interactive/2012/may/18/costa-concordia-salvage-interactive">http://www.guardian.co.uk/world/interactive/2012/may/18/costa-concordia-salvage-interactive</a></cite></p>\n\n<p>The article has changed <var>0</var> times. There is an <a class="feed rss" rel="alternate" href="https://www.newssniffer.co.uk/articles/518330.rss">RSS feed of changes</a> available.</p>\n\n<table>\n<tbody><tr>\n  <th class="version">Version</th>\n  <th class="created_at">Created at</th>\n\t<th class="period">Period</th>\n  <th class="title">Title</th>\n</tr>\n</tbody></table>\n\n\t</div>\n\n\t<footer>\n\t\t<div id="footer">\n\t\t\t\t<p>Currently monitoring 1,792,722 news articles \n\t\t\t\twith 3,829,373 different versions.</p>\n\t\t</div>\n\t</footer>\n\n\n</body></html> """

In [99]:
display(HTML(t))

Version,Created at,Period,Title
